In [2]:
import pandas as pd

In [24]:
df = pd.read_csv('../../data/landing/match_results_23.csv', index_col=0)

In [25]:
df.columns

Index(['id', 'providerId', 'utcStartTime', 'status', 'compSeason.id',
       'compSeason.providerId', 'compSeason.name', 'compSeason.shortName',
       'compSeason.currentRoundNumber', 'round.id', 'round.providerId',
       'round.abbreviation', 'round.name', 'round.roundNumber', 'home.team.id',
       'home.team.providerId', 'home.team.name', 'home.team.abbreviation',
       'home.team.nickname', 'home.team.teamType', 'home.team.club.id',
       'home.team.club.providerId', 'home.team.club.name',
       'home.team.club.abbreviation', 'home.team.club.nickname',
       'home.score.goals', 'home.score.behinds', 'home.score.totalScore',
       'home.score.superGoals', 'away.team.id', 'away.team.providerId',
       'away.team.name', 'away.team.abbreviation', 'away.team.nickname',
       'away.team.teamType', 'away.team.club.id', 'away.team.club.providerId',
       'away.team.club.name', 'away.team.club.abbreviation',
       'away.team.club.nickname', 'away.score.goals', 'away.score.behinds

In [26]:
df = df[['round.roundNumber',
    'home.team.name', 'home.score.goals', 'home.score.behinds', 'home.score.totalScore',
    'away.team.name', 'away.score.goals', 'away.score.behinds', 'away.score.totalScore',
    ]]

In [27]:
df.rename(columns={
    'round.roundNumber': 'round_number',
    'home.team.name': 'home_team', 
    'home.score.goals': 'home_team_goals', 
    'home.score.behinds': 'home_team_behinds', 
    'home.score.totalScore': 'home_team_total',
    'away.team.name': 'away_team', 
    'away.score.goals': 'away_team_goals', 
    'away.score.behinds': 'away_team_behinds', 
    'away.score.totalScore': 'away_team_total',
}, inplace=True)

In [28]:
df = df.query('round_number <= 24')

In [21]:
matches = pd.read_parquet('../../data/raw/2023_match_data.parquet')

In [22]:
matches

,match_id,round_number,home_team_name,away_team_name
0,10007,1,Richmond,Carlton
1,10001,1,Geelong Cats,Collingwood
2,10005,1,North Melbourne,West Coast Eagles
3,10006,1,Port Adelaide,Brisbane Lions
4,10004,1,Melbourne,Western Bulldogs
...,...,...,...,...
202,10201,24,Geelong Cats,Western Bulldogs
203,10206,24,West Coast Eagles,Adelaide Crows
204,10204,24,Port Adelaide,Richmond
205,10205,24,Sydney Swans,Melbourne


In [34]:
df = pd.merge(df, matches,
         left_on=['round_number', 'home_team' , 'away_team'],
         right_on=['round_number', 'home_team_name', 'away_team_name'],
         )

In [35]:
df.columns

Index(['round_number', 'home_team', 'home_team_goals', 'home_team_behinds',
       'home_team_total', 'away_team', 'away_team_goals', 'away_team_behinds',
       'away_team_total', 'match_id', 'home_team_name', 'away_team_name'],
      dtype='object')

In [36]:
df = df[[
    'match_id',
    'round_number', 
    'home_team', 'home_team_goals', 'home_team_behinds', 'home_team_total',
    'away_team', 'away_team_goals', 'away_team_behinds', 'away_team_total'
]]

In [37]:
df['match_margin'] = abs(df['home_team_total'] - df['away_team_total'])

In [39]:
def get_winner(row):

    if row['home_team_total'] > row['away_team_total']:
        return row['home_team']
    elif row['home_team_total'] < row['away_team_total']:
        return row['away_team']
    else:
        return 'Draw'

In [41]:
df['match_winner'] = df.apply(get_winner, axis=1)

In [43]:
df

,match_id,round_number,home_team,home_team_goals,home_team_behinds,home_team_total,away_team,away_team_goals,away_team_behinds,away_team_total,match_margin,match_winner
0,10007,1,Richmond,8.0,10.0,58.0,Carlton,8.0,10.0,58.0,0.0,Draw
1,10001,1,Geelong Cats,16.0,7.0,103.0,Collingwood,19.0,11.0,125.0,22.0,Collingwood
2,10005,1,North Melbourne,12.0,15.0,87.0,West Coast Eagles,12.0,10.0,82.0,5.0,North Melbourne
3,10006,1,Port Adelaide,18.0,18.0,126.0,Brisbane Lions,11.0,6.0,72.0,54.0,Port Adelaide
4,10004,1,Melbourne,17.0,13.0,115.0,Western Bulldogs,9.0,11.0,65.0,50.0,Melbourne
...,...,...,...,...,...,...,...,...,...,...,...,...
202,10201,24,Geelong Cats,11.0,13.0,79.0,Western Bulldogs,16.0,8.0,104.0,25.0,Western Bulldogs
203,10206,24,West Coast Eagles,12.0,6.0,78.0,Adelaide Crows,17.0,21.0,123.0,45.0,Adelaide Crows
204,10204,24,Port Adelaide,13.0,16.0,94.0,Richmond,8.0,15.0,63.0,31.0,Port Adelaide
205,10205,24,Sydney Swans,7.0,14.0,56.0,Melbourne,11.0,11.0,77.0,21.0,Melbourne


In [42]:
df.to_parquet('../../data/raw/match_results.parquet')